In [1]:
import time
import csv
import pandas as pd
import argparse
import numpy as np
import random
from multiprocessing import *
import mplleaflet
import matplotlib.pyplot as plt
from collections import deque

from lib.OsrmEngine import *
from lib.Agents import *
from lib.Demand import *

In [2]:
exe_loc = './osrm-backend-5.6.0/build/osrm-routed'
map_loc = './osrm-backend-5.6.0/greater-london-latest.osrm'

osrm = OsrmEngine(exe_loc, map_loc)
osrm.start_server()

'The routing server "http://0.0.0.0:5000" starts running'

In [3]:
olat = 0.089653
olng = 51.373206
dlat = 0.089282
dlng = 51.350675

N = 100
stime = time.time()
for i in range(N):
    o1 = random.uniform(-0.01, 0.01)
    o2 = random.uniform(-0.01, 0.01)
    d1 = random.uniform(-0.01, 0.01)
    d2 = random.uniform(-0.01, 0.01)
    out = osrm.get_routing(olat+o1, olng+o2, dlat+d1, dlng+d2)
etime = time.time()
print("Avarage running time per request: %.3fms" % ((etime-stime)*1000/N) )

Avarage running time per request: 5.017ms


In [4]:
out = osrm.get_routing(olat, olng, dlat, dlng)
steps = out['legs'][0]['steps']
steps

[{'distance': 134.6,
  'duration': 33.1,
  'geometry': {'coordinates': [[0.089603, 51.373194],
    [0.089608, 51.373185],
    [0.089751, 51.37296],
    [0.089842, 51.372817],
    [0.090346, 51.372133],
    [0.09035, 51.372081]],
   'type': 'LineString'},
  'intersections': [{'bearings': [161],
    'entry': [True],
    'location': [0.089603, 51.373194],
    'out': 0}],
  'maneuver': {'bearing_after': 161,
   'bearing_before': 0,
   'location': [0.089603, 51.373194],
   'type': 'depart'},
  'mode': 'driving',
  'name': 'Station Approach',
  'weight': 33.1},
 {'distance': 125.3,
  'duration': 11.9,
  'geometry': {'coordinates': [[0.09035, 51.372081],
    [0.090681, 51.371981],
    [0.091107, 51.371866],
    [0.091603, 51.371806],
    [0.092071, 51.3718]],
   'type': 'LineString'},
  'intersections': [{'bearings': [120, 300, 345],
    'entry': [True, True, False],
    'in': 2,
    'location': [0.09035, 51.372081],
    'out': 0},
   {'bearings': [105, 195, 300],
    'entry': [True, True, Fa

In [5]:
out = osrm.get_distance_duration(olat, olng, dlat, dlng)
out

(2830.1, 285.9)

In [34]:
class Model(object):
    """
    Model is the class for the AMoD system
    Attributes:
        D: average arrival interval (sec)
        demand: demand matrix
        V: number of vehicles
        K: capacity of vehicles
        vehs: the list of vehicles
        N: number of requests
        reqs: the list of requests
        queue: requests in the queue
    """ 
    def __init__(self, D, demand, V=2, K=4):
        self.D = D
        self.demand = demand
        self.V = V
        self.K = K
        self.vehs = []
        for i in range(V):
            self.vehs.append(Veh(i, k=K))
        self.N = 0
        self.reqs = []
        self.queue = deque([])
        
    def generate_request(self):
        dt = self.D * np.random.exponential()
        rand = np.random.rand()
        for d in demand:
            if d[4] > rand:
                req = Req(0 if self.N == 0 else self.reqs[-1].id+1,
                          dt if self.N == 0 else self.reqs[-1].t+dt,
                          d[1], d[0], d[3], d[2])
                break
        self.N += 1
        return req
        
    def generate_requests_to_time(self, t):
        if self.N == 0:
            req = self.generate_request()
            self.reqs.append(req)
        while self.reqs[-1].t <= t:
            req = self.generate_request()
            self.queue.append(self.reqs[-1])
            self.reqs.append(req)
        assert self.N == len(self.reqs)
        
    def dispatch_at_time(self, t):
        for v in self.vehs:
            v.move_to_time(t)
            print(v)
        self.generate_requests_to_time(t)
        print(self)
        
    def __str__(self):
        str = "AMoD system: %d vehicles of capacity %d; %.1f trips/h" % (self.V, self.K, 3600/self.D)
        str += "\n  has {0} requests, in which {1} in queue".format( self.N, len(self.queue) )
        for r in self.queue:
            str += "\n    " + r.__str__()
        return str

In [36]:
model = Model(30, demand)
model.dispatch_at_time(30)
model.dispatch_at_time(60)
model.dispatch_at_time(90)

veh 0 at (0.0896530, 51.3732060) when t = 30.000; occupancy = 0/4
  has 0 job(s), distance = 0.0, cost = 0.0
veh 1 at (0.0896530, 51.3732060) when t = 30.000; occupancy = 0/4
  has 0 job(s), distance = 0.0, cost = 0.0
AMoD system: 2 vehicles of capacity 4; 120.0 trips/h
  has 1 requests, in which 0 in queue
veh 0 at (0.0896530, 51.3732060) when t = 60.000; occupancy = 0/4
  has 0 job(s), distance = 0.0, cost = 0.0
veh 1 at (0.0896530, 51.3732060) when t = 60.000; occupancy = 0/4
  has 0 job(s), distance = 0.0, cost = 0.0
AMoD system: 2 vehicles of capacity 4; 120.0 trips/h
  has 3 requests, in which 2 in queue
    req 0 from (0.0687732, 51.3715380) to (0.0603542, 51.4061583) at t = 31.621
    req 1 from (0.0525978, 51.3391907) to (0.1005414, 51.3756921) at t = 50.092
veh 0 at (0.0896530, 51.3732060) when t = 90.000; occupancy = 0/4
  has 0 job(s), distance = 0.0, cost = 0.0
veh 1 at (0.0896530, 51.3732060) when t = 90.000; occupancy = 0/4
  has 0 job(s), distance = 0.0, cost = 0.0
AMoD

In [8]:
veh = Veh(0)
req = Req(0, 0)
veh.add_pickup_job(osrm, req)
veh.add_dropoff_job(osrm, req)
print(veh)
done = veh.move_to_time(300)
print(done)
print(veh)

veh 0 at (0.0896530, 51.3732060) when t = 0.000; occupancy = 0/4
  has 2 job(s), distance = 6431.9, cost = 721.2
    pickup: req 0 at (0.1156620, 51.3742820)
    dropoff: req 0 at (0.0892820, 51.3506750)
[(0, 1, 277.4)]
veh 0 at (0.1149032, 51.3732300) when t = 300.000; occupancy = 1/4
  has 1 job(s), distance = 4097.1, cost = 421.2
    dropoff: req 0 at (0.0892820, 51.3506750)


In [9]:
fig = plt.figure(figsize=(16,10))
veh.draw()
mplleaflet.display(fig)

In [ ]:
osrm.kill_server()